In [16]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [17]:
dataset = 'NJH2024'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/media-2.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for NJH2024 ...
Raw data loaded.


,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,Sub10_MICT_Post,Sub10_HIIT_Pre,...,Protein names,Gene names,Sequence window,Phospho (STY) Probabilities,id,Unique identifier,PhosphoSitePlus window,PhosphoSitePlus kinase,PhosphoSitePlus kinase uniprot,Number of Phospho (STY)
0,19.386229,19.018353,18.720772,19.433043,18.761337,18.81024,16.331530,15.998238,16.188589,15.640754,...,Serpin B6,SERPINB6,TWVAEKTEGKIAELLSPGSVDPLTRLVLVNA,IAELLS(1)PGSVDPLTR,0,UID1,GKIAELLsPGSVDPL,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Serpin B6,SERPINB6,AEKTEGKIAELLSPGSVDPLTRLVLVNAVYF,IAELLS(0.454)PGS(0.541)VDPLT(0.005)R,1,UID2,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,19.020823,18.392426,18.674765,18.702187,...,Serpin B6,SERPINB6,VSLPRFKLEESYDMESVLRNLGMTDAFELGK,LEESYDMES(1)VLR,2,UID3,NaN,NaN,NaN,1
3,NaN,NaN,NaN,NaN,NaN,NaN,17.989483,17.280300,17.360332,17.595287,...,Vigilin,HDLBP,_____MSSVAVLTQESFAEHRSGLVPQQIKV,SSVAVLTQES(1)FAEHR,3,UID4,VAVLtQEsFAEHRSG,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Vigilin,HDLBP,RSGLVPQQIKVATLNSEEESDPPTYKDAFPP,VAT(0.016)LNS(0.979)EEES(0.004)DPPTYK,4,UID5,IkVAtLNsEEEsDPP,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Dynein heavy chain 6, axonemal",DNAH6,NLKVVSYENINKNDYYTISQRAVTHIYNEDI,VVS(0.003)Y(0.004)ENINKNDY(0.763)Y(0.763)T(0.7...,14369,UID42485,NaN,NaN,NaN,3
14927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Synaptopodin 2-like protein,SYNPO2L,LQPGSLRSPPDSEAYYGETDSDADGPATQEK,SPPDSEAY(0.111)Y(0.842)GET(0.069)DS(0.977)DADG...,14376,UID42492,PPDSEAYyGEtDsDA,NaN,NaN,2;3
14928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Myozenin-1,MYOZ1,GRGGSQAGGSGSAGQYGSDQQHHLGSGSGAG,GGSQAGGSGS(0.006)AGQY(0.997)GS(0.997)DQQHHLGSG...,14380,UID42496,GSGSAGQyGSDQQHH,NaN,NaN,1;2;3
14929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Synaptopodin-2,SYNPO2,RNMTSPIADFPAPPPYSAVTPPPDAFSRGVS,NMT(0.009)S(0.029)PIADFPAPPPY(0.899)S(0.066)AV...,14388,UID42504,DFPAPPPysAVtPPP,NaN,NaN,1;2;3


In [18]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization prob'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Positions within proteins'].str.contains(';', na=False)]
data = data[~data['Gene names'].str.contains(';', na=False)]
data

,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,Sub10_MICT_Post,Sub10_HIIT_Pre,...,Protein names,Gene names,Sequence window,Phospho (STY) Probabilities,id,Unique identifier,PhosphoSitePlus window,PhosphoSitePlus kinase,PhosphoSitePlus kinase uniprot,Number of Phospho (STY)
258,18.701374,18.322325,17.900772,18.821119,18.142855,18.175432,NaN,NaN,NaN,NaN,...,NaN,ALG13,KDSLKEDIQKADLVISHAGKGAGSCLETLEK,ADLVIS(1)HAGK,381,UID382,NaN,NaN,NaN,1
1034,NaN,NaN,NaN,NaN,NaN,NaN,15.324883,14.986598,14.852432,14.572641,...,NaN,NaN,LRGNTSHASSSQQVPSPTGRNPTGAPTSLAA,GNTSHASSSQQVPS(0.992)PT(0.008)GR,1365,UID1365,NaN,NaN,NaN,1
1035,NaN,NaN,NaN,NaN,NaN,NaN,18.870840,18.465231,18.556334,18.677347,...,NaN,NaN,SQERPLTQADLGRQQSHQAQEETPQPGDAGK,QQS(1)HQAQEETPQPGDAGK,1366,UID1366,NaN,NaN,NaN,1
1066,16.634102,16.204361,15.936983,18.434246,15.577045,15.910057,NaN,NaN,NaN,NaN,...,NaN,NaN,FRGESDEPAPQVQPESPETVSPKGSEGLPPP,GESDEPAPQVQPES(0.993)PET(0.006)VS(0.001)PK,1424,UID1424,NaN,NaN,NaN,1
1101,18.311810,17.588129,16.907713,18.782736,17.295177,17.458920,17.958439,17.500048,17.302513,17.711962,...,NaN,NaN,ASQEVTIAVNRGERLSLDKLECTDQETESEN,GERLS(1)LDK,1478,UID1478,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14915,16.990812,16.997885,16.629841,17.377974,16.432819,16.528286,15.805668,15.736032,15.461351,15.841171,...,Muscle-related coiled-coil protein,MURC,DIIARSESLGPISELYSDELSEPEHEAARPV,SESLGPIS(0.002)ELY(0.98)S(0.021)DELS(0.997)EPE...,14334,UID42450,NaN,NaN,NaN,2;3
14919,18.623745,18.352594,17.969725,18.800192,18.444593,18.417614,NaN,NaN,NaN,NaN,...,Metallo-beta-lactamase domain-containing prote...,MBLAC2,FNTFGAERLFRLASNYISKAGICHKVSTFAM,LFRLAS(1)NY(1)IS(1)K,14342,UID42458,NaN,NaN,NaN,3
14920,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,Uncharacterized protein C3orf20,C3orf20,_____MSYIKSNLELYQQYTAMAPKLLARIS,SNLELY(1)QQY(1)T(1)AMAPK,14355,UID42471,NaN,NaN,NaN,3
14921,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,Uncharacterized protein C3orf20,C3orf20,__MSYIKSNLELYQQYTAMAPKLLARISKLL,SNLELY(1)QQY(1)T(1)AMAPK,14356,UID42472,NaN,NaN,NaN,3


In [19]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,Sub10_MICT_Post,Sub10_HIIT_Pre,...,Protein names,Gene names,Sequence window,Phospho (STY) Probabilities,id,Unique identifier,PhosphoSitePlus window,PhosphoSitePlus kinase,PhosphoSitePlus kinase uniprot,Number of Phospho (STY)
258,18.701374,18.322325,17.900772,18.821119,18.142855,18.175432,NaN,NaN,NaN,NaN,...,NaN,ALG13,KDSLKEDIQKADLVISHAGKGAGSCLETLEK,ADLVIS(1)HAGK,381,UID382,NaN,NaN,NaN,1
1034,NaN,NaN,NaN,NaN,NaN,NaN,15.324883,14.986598,14.852432,14.572641,...,NaN,NaN,LRGNTSHASSSQQVPSPTGRNPTGAPTSLAA,GNTSHASSSQQVPS(0.992)PT(0.008)GR,1365,UID1365,NaN,NaN,NaN,1
1035,NaN,NaN,NaN,NaN,NaN,NaN,18.870840,18.465231,18.556334,18.677347,...,NaN,NaN,SQERPLTQADLGRQQSHQAQEETPQPGDAGK,QQS(1)HQAQEETPQPGDAGK,1366,UID1366,NaN,NaN,NaN,1
1066,16.634102,16.204361,15.936983,18.434246,15.577045,15.910057,NaN,NaN,NaN,NaN,...,NaN,NaN,FRGESDEPAPQVQPESPETVSPKGSEGLPPP,GESDEPAPQVQPES(0.993)PET(0.006)VS(0.001)PK,1424,UID1424,NaN,NaN,NaN,1
1101,18.311810,17.588129,16.907713,18.782736,17.295177,17.458920,17.958439,17.500048,17.302513,17.711962,...,NaN,NaN,ASQEVTIAVNRGERLSLDKLECTDQETESEN,GERLS(1)LDK,1478,UID1478,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14915,16.990812,16.997885,16.629841,17.377974,16.432819,16.528286,15.805668,15.736032,15.461351,15.841171,...,Muscle-related coiled-coil protein,MURC,DIIARSESLGPISELYSDELSEPEHEAARPV,SESLGPIS(0.002)ELY(0.98)S(0.021)DELS(0.997)EPE...,14334,UID42450,NaN,NaN,NaN,2;3
14919,18.623745,18.352594,17.969725,18.800192,18.444593,18.417614,NaN,NaN,NaN,NaN,...,Metallo-beta-lactamase domain-containing prote...,MBLAC2,FNTFGAERLFRLASNYISKAGICHKVSTFAM,LFRLAS(1)NY(1)IS(1)K,14342,UID42458,NaN,NaN,NaN,3
14920,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,Uncharacterized protein C3orf20,C3orf20,MSYIKSNLELYQQYTAMAPKLLARIS,SNLELY(1)QQY(1)T(1)AMAPK,14355,UID42471,NaN,NaN,NaN,3
14921,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,Uncharacterized protein C3orf20,C3orf20,MSYIKSNLELYQQYTAMAPKLLARISKLL,SNLELY(1)QQY(1)T(1)AMAPK,14356,UID42472,NaN,NaN,NaN,3


In [20]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    

In [21]:
data = match_seq_to_genename(data, 'Sequence window')

KDSLKEDIQKADLVISHAGKGAGSCLETLEK
LRGNTSHASSSQQVPSPTGRNPTGAPTSLAA
SQERPLTQADLGRQQSHQAQEETPQPGDAGK
FRGESDEPAPQVQPESPETVSPKGSEGLPPP
ASQEVTIAVNRGERLSLDKLECTDQETESEN
TDPLSEPDPPARDAASPHQDGAPGPGPRGGR
GDADSLFTSLSERAASPERPPRKAHVGPREE
GSGEDRAGSSLSFSLSGSPGSRRSTSRLDSL
GEDRAGSSLSFSLSGSPGSRRSTSRLDSLSR
TLSPSLSRASGLGRESPDSRLSLGRSCLEEW
PSLSRASGLGRESPDSRLSLGRSCLEEWDDG
VVSSLSYRKRLTLKDSIGGTGDADSLFTSLS
KDSIGGTGDADSLFTSLSERAASPERPPRKA
SSLSKEAPGVEERPSSVVSSLSYRKRLTLKD
APGVEERPSSVVSSLSYRKRLTLKDSIGGTG
SSHSCSSVKKGPAAESEETGQTHRPSRKEEK
PDSEEDDRKSIQSARSAQLDPPERAASIRRS
QLDPPERAASIRRSASAADVSRSRSGRKSRS
HPSLARSLSVPPRPRSSASAVDEPPSSSVRS
LARSLSVPPRPRSSASAVDEPPSSSVRSVSR
SYKPVPFQRSSLEPDSEEDDRKSIQSARSAQ
SRRSTSRLDSLSRTLSPSLSRASGLGRESPD
RSTSRLDSLSRTLSPSLSRASGLGRESPDSR
TSRLDSLSRTLSPSLSRASGLGRESPDSRLS
SQGYVSENGHDHREGSMGEDTLPDQGPPYER
GGGLQGGVGALKRSSSMFIPQLLTSIDARPT
DRLVRYPRIRLERSTSYPTQPRSERGSPTED
KAGSPAKPEAPKKQRSPAPPEELVTPGSTGP
PPTSPIPPETSQRARSPAPTLAMNGPGAATA
GALEGSSKEGSGPSRSPQPPTSPIPPETSQR
SKEGSGPSRSPQPPTSPIPPETSQRARSPAP
WEQKEKED

In [22]:
data

,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,Sub10_MICT_Post,Sub10_HIIT_Pre,...,Gene names,Sequence window,Phospho (STY) Probabilities,id,Unique identifier,PhosphoSitePlus window,PhosphoSitePlus kinase,PhosphoSitePlus kinase uniprot,Number of Phospho (STY),GeneName
258,18.701374,18.322325,17.900772,18.821119,18.142855,18.175432,NaN,NaN,NaN,NaN,...,ALG13,KDSLKEDIQKADLVISHAGKGAGSCLETLEK,ADLVIS(1)HAGK,381,UID382,NaN,NaN,NaN,1,NaN
1034,NaN,NaN,NaN,NaN,NaN,NaN,15.324883,14.986598,14.852432,14.572641,...,NaN,LRGNTSHASSSQQVPSPTGRNPTGAPTSLAA,GNTSHASSSQQVPS(0.992)PT(0.008)GR,1365,UID1365,NaN,NaN,NaN,1,NaN
1035,NaN,NaN,NaN,NaN,NaN,NaN,18.870840,18.465231,18.556334,18.677347,...,NaN,SQERPLTQADLGRQQSHQAQEETPQPGDAGK,QQS(1)HQAQEETPQPGDAGK,1366,UID1366,NaN,NaN,NaN,1,NaN
1066,16.634102,16.204361,15.936983,18.434246,15.577045,15.910057,NaN,NaN,NaN,NaN,...,NaN,FRGESDEPAPQVQPESPETVSPKGSEGLPPP,GESDEPAPQVQPES(0.993)PET(0.006)VS(0.001)PK,1424,UID1424,NaN,NaN,NaN,1,NaN
1101,18.311810,17.588129,16.907713,18.782736,17.295177,17.458920,17.958439,17.500048,17.302513,17.711962,...,NaN,ASQEVTIAVNRGERLSLDKLECTDQETESEN,GERLS(1)LDK,1478,UID1478,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14915,16.990812,16.997885,16.629841,17.377974,16.432819,16.528286,15.805668,15.736032,15.461351,15.841171,...,MURC,DIIARSESLGPISELYSDELSEPEHEAARPV,SESLGPIS(0.002)ELY(0.98)S(0.021)DELS(0.997)EPE...,14334,UID42450,NaN,NaN,NaN,2;3,CAVIN4
14919,18.623745,18.352594,17.969725,18.800192,18.444593,18.417614,NaN,NaN,NaN,NaN,...,MBLAC2,FNTFGAERLFRLASNYISKAGICHKVSTFAM,LFRLAS(1)NY(1)IS(1)K,14342,UID42458,NaN,NaN,NaN,3,MBLAC2
14920,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,C3orf20,MSYIKSNLELYQQYTAMAPKLLARIS,SNLELY(1)QQY(1)T(1)AMAPK,14355,UID42471,NaN,NaN,NaN,3,C3orf20
14921,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,C3orf20,MSYIKSNLELYQQYTAMAPKLLARISKLL,SNLELY(1)QQY(1)T(1)AMAPK,14356,UID42472,NaN,NaN,NaN,3,C3orf20


In [23]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'
data

,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,Sub10_MICT_Post,Sub10_HIIT_Pre,...,Sequence window,Phospho (STY) Probabilities,id,Unique identifier,PhosphoSitePlus window,PhosphoSitePlus kinase,PhosphoSitePlus kinase uniprot,Number of Phospho (STY),GeneName,Phosphosite
258,18.701374,18.322325,17.900772,18.821119,18.142855,18.175432,NaN,NaN,NaN,NaN,...,KDSLKEDIQKADLVISHAGKGAGSCLETLEK,ADLVIS(1)HAGK,381,UID382,NaN,NaN,NaN,1,NaN,S(74)
1034,NaN,NaN,NaN,NaN,NaN,NaN,15.324883,14.986598,14.852432,14.572641,...,LRGNTSHASSSQQVPSPTGRNPTGAPTSLAA,GNTSHASSSQQVPS(0.992)PT(0.008)GR,1365,UID1365,NaN,NaN,NaN,1,NaN,S(1029)
1035,NaN,NaN,NaN,NaN,NaN,NaN,18.870840,18.465231,18.556334,18.677347,...,SQERPLTQADLGRQQSHQAQEETPQPGDAGK,QQS(1)HQAQEETPQPGDAGK,1366,UID1366,NaN,NaN,NaN,1,NaN,S(1382)
1066,16.634102,16.204361,15.936983,18.434246,15.577045,15.910057,NaN,NaN,NaN,NaN,...,FRGESDEPAPQVQPESPETVSPKGSEGLPPP,GESDEPAPQVQPES(0.993)PET(0.006)VS(0.001)PK,1424,UID1424,NaN,NaN,NaN,1,NaN,S(866)
1101,18.311810,17.588129,16.907713,18.782736,17.295177,17.458920,17.958439,17.500048,17.302513,17.711962,...,ASQEVTIAVNRGERLSLDKLECTDQETESEN,GERLS(1)LDK,1478,UID1478,NaN,NaN,NaN,1,NaN,S(932)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14915,16.990812,16.997885,16.629841,17.377974,16.432819,16.528286,15.805668,15.736032,15.461351,15.841171,...,DIIARSESLGPISELYSDELSEPEHEAARPV,SESLGPIS(0.002)ELY(0.98)S(0.021)DELS(0.997)EPE...,14334,UID42450,NaN,NaN,NaN,2;3,CAVIN4,Y(310)
14919,18.623745,18.352594,17.969725,18.800192,18.444593,18.417614,NaN,NaN,NaN,NaN,...,FNTFGAERLFRLASNYISKAGICHKVSTFAM,LFRLAS(1)NY(1)IS(1)K,14342,UID42458,NaN,NaN,NaN,3,MBLAC2,Y(247)
14920,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,MSYIKSNLELYQQYTAMAPKLLARIS,SNLELY(1)QQY(1)T(1)AMAPK,14355,UID42471,NaN,NaN,NaN,3,C3orf20,Y(11)
14921,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,15.923119,16.220360,...,MSYIKSNLELYQQYTAMAPKLLARISKLL,SNLELY(1)QQY(1)T(1)AMAPK,14356,UID42472,NaN,NaN,NaN,3,C3orf20,Y(14)


In [24]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite', 'GeneName'] + [col for col in data.columns if col.startswith('Sub')]
data = data[keepcols]
data

,Phosphosite,GeneName,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,...,Sub8_MICT_Post,Sub8_HIIT_Pre,Sub8_HIIT_Mid,Sub8_HIIT_Post,Sub9_MICT_Pre,Sub9_MICT_Mid,Sub9_MICT_Post,Sub9_HIIT_Pre,Sub9_HIIT_Mid,Sub9_HIIT_Post
258,S(74),NaN,18.701374,18.322325,17.900772,18.821119,18.142855,18.175432,NaN,NaN,...,18.684660,18.931770,18.303495,18.072165,NaN,NaN,NaN,NaN,NaN,NaN
1034,S(1029),NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.324883,14.986598,...,14.943294,14.803223,14.191831,13.050682,15.738647,15.811425,16.188646,15.688769,14.724461,15.121493
1035,S(1382),NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.870840,18.465231,...,17.343948,18.335949,17.029179,17.191135,16.590544,16.088457,16.366407,16.009457,15.091146,15.607475
1066,S(866),NaN,16.634102,16.204361,15.936983,18.434246,15.577045,15.910057,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1101,S(932),NaN,18.311810,17.588129,16.907713,18.782736,17.295177,17.458920,17.958439,17.500048,...,NaN,NaN,NaN,NaN,18.035315,17.443542,17.456194,17.185921,17.092901,16.768236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14915,Y(310),CAVIN4,16.990812,16.997885,16.629841,17.377974,16.432819,16.528286,15.805668,15.736032,...,15.228556,15.554379,15.256283,15.209530,17.575184,17.129383,17.345509,17.060337,17.075563,17.066761
14919,Y(247),MBLAC2,18.623745,18.352594,17.969725,18.800192,18.444593,18.417614,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14920,Y(11),C3orf20,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,...,16.250910,16.433519,15.831455,15.672149,NaN,NaN,NaN,NaN,NaN,NaN
14921,Y(14),C3orf20,NaN,NaN,NaN,NaN,NaN,NaN,16.585960,15.762564,...,16.250910,16.433519,15.831455,15.672149,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
Sub_columns = [col for col in data.columns if 'Sub' in col]
data[Sub_columns] = data[Sub_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_2969/3338010094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Sub_columns] = data[Sub_columns].apply(pd.to_numeric, errors='coerce')


In [26]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.DataFrame>
    
    out:
    ====
    dataset: <pd.DataFrame> with log2 transformed values
    '''
    cols_to_transform = [col for col in dataset.columns if col.startswith('Sub')]
    dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])
    print('Data has been log2 transformed.')
    return dataset

In [27]:
# Apply the log2 transformation
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}")
data

Data has been log2 transformed.
DataFrame after log2 transformation:
      Phosphosite GeneName  Sub1_MICT_Pre  Sub1_MICT_Mid  Sub1_MICT_Post  \
258         S(74)      NaN       4.225072       4.195531        4.161950   
1034      S(1029)      NaN            NaN            NaN             NaN   
1035      S(1382)      NaN            NaN            NaN             NaN   
1066       S(866)      NaN       4.056072       4.018310        3.994307   
1101       S(932)      NaN       4.194702       4.136530        4.079610   
...           ...      ...            ...            ...             ...   
14915      Y(310)   CAVIN4       4.086683       4.087283        4.055702   
14919      Y(247)   MBLAC2       4.219071       4.197912        4.167496   
14920       Y(11)  C3orf20            NaN            NaN             NaN   
14921       Y(14)  C3orf20            NaN            NaN             NaN   
14928      Y(127)    MYOZ1            NaN            NaN             NaN   

       Sub1_HIIT_P

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_2969/3362102874.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])


,Phosphosite,GeneName,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,...,Sub8_MICT_Post,Sub8_HIIT_Pre,Sub8_HIIT_Mid,Sub8_HIIT_Post,Sub9_MICT_Pre,Sub9_MICT_Mid,Sub9_MICT_Post,Sub9_HIIT_Pre,Sub9_HIIT_Mid,Sub9_HIIT_Post
258,S(74),NaN,4.225072,4.195531,4.161950,4.234281,4.181330,4.183918,NaN,NaN,...,4.223782,4.242737,4.194047,4.175697,NaN,NaN,NaN,NaN,NaN,NaN
1034,S(1029),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.937804,3.905601,...,3.901426,3.887839,3.826989,3.706053,3.976240,3.982896,4.016910,3.971660,3.880143,3.918529
1035,S(1382),NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.238087,4.206739,...,4.116360,4.196603,4.089937,4.103593,4.052289,4.007954,4.032666,4.000852,3.915631,3.964165
1066,S(866),NaN,4.056072,4.018310,3.994307,4.204317,3.961350,3.991867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1101,S(932),NaN,4.194702,4.136530,4.079610,4.231335,4.112298,4.125892,4.166590,4.129287,...,NaN,NaN,NaN,NaN,4.172753,4.124621,4.125667,4.103155,4.095325,4.067659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14915,Y(310),CAVIN4,4.086683,4.087283,4.055702,4.119188,4.038508,4.046865,3.982370,3.976000,...,3.928707,3.959249,3.931332,3.926904,4.135468,4.098401,4.116490,4.092574,4.093861,4.093117
14919,Y(247),MBLAC2,4.219071,4.197912,4.167496,4.232675,4.205126,4.203014,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14920,Y(11),C3orf20,NaN,NaN,NaN,NaN,NaN,NaN,4.051891,3.978430,...,4.022449,4.038570,3.984722,3.970131,NaN,NaN,NaN,NaN,NaN,NaN
14921,Y(14),C3orf20,NaN,NaN,NaN,NaN,NaN,NaN,4.051891,3.978430,...,4.022449,4.038570,3.984722,3.970131,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_2969/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


Phosphosite IDs created.
Phosphosite IDs created.


In [29]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data



In [32]:
data = clean_phosID_col(data)
data = data[['phosphosite_ID'] + Sub_columns]
print("After cleaning phosphosite_ID column:")
data

There are no phosphosites with multiple measurements
     phosphosite_ID  Sub1_MICT_Pre  Sub1_MICT_Mid  Sub1_MICT_Post  \
0       AAK1_S(846)            NaN            NaN             NaN   
1       AATF_S(203)       4.182080       4.105629        4.098613   
2       ABRA_S(153)       4.034639       4.019976        3.958636   
3       ABRA_S(156)       4.178045       4.140621        4.110844   
4       ABRA_S(188)       4.256687       4.262347        4.239940   
...             ...            ...            ...             ...   
1698  ZNF652_S(197)            NaN            NaN             NaN   
1699  ZNF777_S(157)       4.047089       4.031574        3.994017   
1700   ZNRF2_S(135)            NaN            NaN             NaN   
1701    ZNRF2_S(19)            NaN            NaN             NaN   
1702   ZRSR2_S(349)            NaN            NaN             NaN   

      Sub1_HIIT_Pre  Sub1_HIIT_Mid  Sub1_HIIT_Post  Sub10_MICT_Pre  \
0               NaN            NaN             N

,phosphosite_ID,Sub1_MICT_Pre,Sub1_MICT_Mid,Sub1_MICT_Post,Sub1_HIIT_Pre,Sub1_HIIT_Mid,Sub1_HIIT_Post,Sub10_MICT_Pre,Sub10_MICT_Mid,Sub10_MICT_Post,...,Sub8_MICT_Post,Sub8_HIIT_Pre,Sub8_HIIT_Mid,Sub8_HIIT_Post,Sub9_MICT_Pre,Sub9_MICT_Mid,Sub9_MICT_Post,Sub9_HIIT_Pre,Sub9_HIIT_Mid,Sub9_HIIT_Post
0,AAK1_S(846),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AATF_S(203),4.182080,4.105629,4.098613,4.221570,4.100034,4.117287,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABRA_S(153),4.034639,4.019976,3.958636,4.062705,3.993289,3.969685,4.130474,4.078846,4.088475,...,3.953452,3.941799,3.914933,3.812701,4.157982,4.102667,4.115789,4.110221,4.057101,4.059191
3,ABRA_S(156),4.178045,4.140621,4.110844,4.168841,4.106081,4.102102,4.407151,4.368985,4.388410,...,4.397364,4.372795,4.340689,4.305072,4.375636,4.358924,4.348296,4.366452,4.346737,4.312861
4,ABRA_S(188),4.256687,4.262347,4.239940,4.275290,4.276904,4.270852,4.168703,4.186195,4.206434,...,4.090357,4.103948,4.059955,4.027753,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,ZNF652_S(197),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,ZNF777_S(157),4.047089,4.031574,3.994017,4.070883,3.989814,3.988916,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1700,ZNRF2_S(135),NaN,NaN,NaN,NaN,NaN,NaN,4.125521,4.076060,4.073543,...,NaN,NaN,NaN,NaN,4.126926,4.083381,4.066080,4.078290,4.064674,4.034586
1701,ZNRF2_S(19),NaN,NaN,NaN,NaN,NaN,NaN,4.100915,4.079529,4.039280,...,NaN,NaN,NaN,NaN,4.084163,4.015154,4.019237,4.026076,3.984039,3.950493


In [33]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/NJH2024.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)     

NJH2024 has been saved to CSV successfully!      phosphosite_ID  Sub1_MICT_Pre  Sub1_MICT_Mid  Sub1_MICT_Post  \
0       AAK1_S(846)            NaN            NaN             NaN   
1       AATF_S(203)       4.182080       4.105629        4.098613   
2       ABRA_S(153)       4.034639       4.019976        3.958636   
3       ABRA_S(156)       4.178045       4.140621        4.110844   
4       ABRA_S(188)       4.256687       4.262347        4.239940   
...             ...            ...            ...             ...   
1698  ZNF652_S(197)            NaN            NaN             NaN   
1699  ZNF777_S(157)       4.047089       4.031574        3.994017   
1700   ZNRF2_S(135)            NaN            NaN             NaN   
1701    ZNRF2_S(19)            NaN            NaN             NaN   
1702   ZRSR2_S(349)            NaN            NaN             NaN   

      Sub1_HIIT_Pre  Sub1_HIIT_Mid  Sub1_HIIT_Post  Sub10_MICT_Pre  \
0               NaN            NaN             NaN       